In [39]:
import pandas as pd
import json
import folium


# https://blog.naver.com/kcchang61/221350672356

# 구단위 지도 그리기
geo_path = './data/hankuk1_geo.json'
geo_str = json.load(open(geo_path, encoding='utf-8')) # 딕셔너리 객체로 리턴


# 서울 동별 인구수, 면적 정보
pop = pd.read_csv('./data/서울동별인구데이터.txt', sep="\t")
# 서울 동별 인구 "합계", "소계" 없애기
pop_1 = pop['자치구'].isin(['합계','소계'])
pop_2 = pop[~pop_1]
pop_3 = pop_2['동'].isin(['합계','소계'])

pop_last = pop_2[~pop_3]
# display(pop_last)


# 동별 중앙값 불러오기
dong = pd.read_csv('./data/행정동별위치정보.csv', encoding='euc-kr')
dong = dong[['행정동명', '경도', '위도']]
dong.rename(columns = {'행정동명': '동'}, inplace=True)
# display(dong)

# 서울 동별 인구수, 면적정보, 중앙값 합치기
all_data = pd.merge(pop_last, dong, how='outer', on='동')
all_data.dropna(axis=0, how='any', inplace=True)
# display(all_data)

# # 지하철 리스트를 데이터프레임 변환
df = pd.read_excel('./data/지하철노선위경도정보2.xlsx') # 데이터 프레임 형성
df1 = df.drop_duplicates()
display(df1)


# # 지역별, 동별 구분 경계선 
map = folium.Map(location=[37.5502, 126.982], zoom_start=13,
                 tiles='Stamen Toner')

fmap=folium.Choropleth(geo_data = geo_str, 
#                        data = all_data['인구밀도'], 
#                        columns = [all_data.index, all_data['인구밀도']], 
                       fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.3
                      ).add_to(map) 


                                  
# 지역 이름을 툴팁으로                                  
fmap.geojson.add_child(
    folium.features.GeoJsonTooltip(['adm_nm'],labels=False) # 해당 지역마다 툴팁으로 마우스오버 작동하게 만듦
)


# 인근 지역 마커 카운터(마커 클러스터)
from folium.plugins import MarkerCluster
marker_cluster = MarkerCluster().add_to(map)

# # 서울 지하철 리스트 마커로 표현하기
for sub, lat, lng in zip(df1.역이름, df1.위도, df1.경도): # zip으로 수행 / 하나씩 묶는 것 
    folium.Marker([lat, lng], popup=sub).add_to(map) # 마커 객체 생성 // 팝업 시 이름이 나오도록 설정 / 서울 지도에 넣는다.
    folium.Marker([lat, lng], popup=sub).add_to(marker_cluster)
    
    
    
# # 동별 써클 그리기 --> 동별 중앙값을 찾은 뒤 거기서 반지름 50을 주자
for dong,lat,lng in zip(all_data.동, all_data.위도, all_data.경도):
    folium.CircleMarker(
      [lat,lng],
      radius=100,
      color='#00ff00', # 테두리
      fill_color='#ff000099', # 원 칼라 // 투명도도 같이 준다.
      popup=dong
    ).add_to(map)
    folium.Marker(
        [lat,lng],
        icon=folium.Icon(color='red')
    ).add_to(map)





for i in range(len(df)):
    x = df.iloc[i][1]
    y = df.iloc[i][2]
    folium.Marker(
        location=[x,y],
        icon=folium.Icon(color='red',icon='ok'),
      ).add_to(marker_cluster)



# Q :보행접근성을 어떻게 평가할 것인가? 
# A :동별 면적을 구하고 내부 지하철역 개수로 그 면적을 나눈다. 즉 면적이 작을수록 보행접근성이 높다!
# How? : 동별 지하철 갯수를 세야 함



    
    
display(map)

,역이름,위도,경도
0,4·19민주묘지역,37.649527,127.013689
1,가락시장역,37.493092,127.118258
3,가산디지털단지역,37.480389,126.882644
5,가양역,37.561429,126.854402
6,가오리역,37.641634,127.016769
...,...,...,...
383,화랑대역,37.619838,127.083526
384,회기역,37.589778,127.058039
387,회현역,37.558760,126.978428
388,효창공원앞역,37.539294,126.961394
